In [8]:
dico_graphe = {}

In [1]:
def TrouveCommunaute(communautes, i):
    ''' Renvoie le numero de la communaute du noeud i'''
    
    return communautes[i]

In [2]:
def QuiDansCommunaute(communautes, C):
    ''' Renvoie une liste des noeuds appartenant la communauté C'''

    commu = []

    for n in range(len(communautes)):
        if communautes[n] == C:
            commu.append(n)
    
    return commu

In [3]:
def TrouveVoisins(dico, i):
    ''' Renvoie une liste des voisins du sommet i'''
    
    l_res = []
    v = dico[i]

    for j in v:
        l_res.append(j[0])

    return l_res

In [4]:
def AjoutDansCommunaute(communautes, i, C):
    ''' Ajoute i à la communauté C dans la liste communautés '''
    # communautes est une liste où chaque élément d'indice i est le numéro de la communauté de i

    communautes[i] = C
    return communautes

In [5]:
def MemeCommunaute(communautes, i, j):
    ''' Renvoie True si i et j sont dans la même communauté, False sinon'''

    if TrouveCommunaute(communautes, i) == TrouveCommunaute(communautes, j):
        return True
    return False

## Calcul du gain de modularité

In [6]:
def SommePoidsGraphe(dico):
    ''' Calcule la somme des poids du graphe (m) '''

    somme = 0
    cles = dico.keys()
    for c in cles:
        voisins = dico[c]
        for v in range(len(voisins)):
            somme += voisins[v][1]
    return somme//2

#m = SommePoidsGraphe(dico_graphe)   # appeler la fonction qu'une seule fois

In [7]:
def SommeInside(dico, communautes, C:int):
    ''' Calcule la somme des poids des arêtes dans la communauté C'''
    
    noeuds_de_C = QuiDansCommunaute(communautes, C)
    somme = 0
    
    for noeud in noeuds_de_C:
        voisins = dico[noeud]
        for v in voisins:
            if TrouveCommunaute(communautes, v[0]) == C:
                somme += v[1]
    
    return somme//2

In [8]:
def SommeTotale(dico, communautes, C:int):
    ''' Calcule la somme des poids des arêtes sortant de C (de tous les sommets appartenant à C)'''
    
    noeuds_de_C = QuiDansCommunaute(communautes, C)
    somme = 0
    
    for noeud in noeuds_de_C:
        voisins = dico[noeud]
        for v in voisins:
            if (v[0] not in noeuds_de_C) or (v[0] > noeud):        # vérifier qu'on a pas déjà compté cette arête
                somme += v[1]
    
    return somme

In [9]:
def Ki(dico, i):
    ''' Calcule la somme des poids des arêtes sortant (entrant) de i'''
    
    return len(dico[i])

In [10]:
def KiInside(dico, communautes, i):
    ''' Calcule la somme des poids des arêtes sortant (entrant) de i, dans sa communauté'''
    
    somme = 0
    tous_voisins = dico[i]

    for v in tous_voisins:
        if MemeCommunaute(communautes, i, v[0]):
            somme += v[1]
    
    return somme

In [27]:
def GainModularite(dico, communautes, m, i:int, j:int):
    ''' Calcule le gain de modularité si on ajoute i à la communauté de j'''

    C = TrouveCommunaute(communautes, j)
    S_in = SommeInside(dico, communautes, C)
    k_i_in = KiInside(dico, communautes, i)
    S_tot = SommeTotale(dico, communautes, C)
    k_i = Ki(dico, i)

    Q = (((S_in + 2*k_i_in)/(2*m)) - ((S_tot + k_i)/(2*m))**2) - ((S_in/(2*m)) - (S_tot/(2*m))**2 - (k_i/(2*m))**2)

    return Q

## Algorithme de Louvain

In [50]:
def Louvain1(dico):
    ''' Première partie de l'algorithme de Louvain '''

    flag_modif = True
    m = SommePoidsGraphe(dico)
    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

   

    while flag_modif == True:
        flag_modif = False          # aucune modification n'a encore été faite pour cette itération
        for i in range(len(communautes)):        # pour chaque noeud i
            max_gain = -1
            max_j = -1
            voisins_de_i = TrouveVoisins(dico, i)
            for j in voisins_de_i:      # pour chaque voisin de i
                gain_mod = GainModularite(dico, communautes, m, i, j)      # calcul du gain de modularité
                print('i:', i, 'j:', j, 'gain:', gain_mod)
                if gain_mod > max_gain:          # /!\ gérer l'égalité
                    max_gain = gain_mod
                    max_j = j
            if max_gain > 0:        # /!\ strict ou pas ?
                flag_modif = True
                communautes = AjoutDansCommunaute(communautes, i, TrouveCommunaute(communautes, max_j))       # ajout de i dans la communauté de j
    
    return communautes

## Tests !

In [13]:
# Le graphe test contient 21 sommets et 34 arêtes

dico_test = {0:[(1,1)],
            1:[(0,1), (2,1)],
            2:[(1,1), (3,1), (4,1)],
            3:[(2,1), (4,1), (8,1)],
            4:[(2,1), (3,1), (5,1), (7,1)],
            5:[(4,1), (6,1), (17,1)],
            6:[(5,1), (7,1), (19,1)],
            7:[(4,1), (6,1), (9,1)],
            8:[(3,1)],
            9:[(7,1), (10,1), (11,1)],
            10:[(9,1), (11,1)],
            11:[(9,1), (10,1), (12,1)],
            12:[(11,1), (14,1), (17,1), (19,1)],
            13:[(14,1), (15,1), (16,1), (19,1)],
            14:[(12,1), (13,1), (15,1)],
            15:[(13,1), (14,1), (16,1), (17,1)],
            16:[(13,1), (15,1), (17,1), (18,1), (20,1)],
            17:[(5,1), (12,1), (15,1), (16,1), (18,1)],
            18:[(16,1), (17,1), (19,1), (20,1)],
            19:[(6,1), (12,1), (13,1), (18,1), (20,1)],
            20:[(16,1), (18,1), (19,1)]
            }

In [44]:
dico_test2 = {0:[(1,1), (2,1), (3,1)],
              1:[(0,1), (2,1), (3,1)],
              2:[(0,1), (1,1), (3,1)],
              3:[(0,1), (1,1), (2,1), (4,1)],
              4:[(5,1), (6,1), (7,1), (3,1)],
              5:[(4,1), (6,1), (7,1)],
              6:[(4,1), (5,1), (7,1)],
              7:[(4,1), (5,1), (6,1)],}

In [14]:
communautes = [0, 0, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5]

In [22]:
print(SommePoidsGraphe(dico_test))

34


In [28]:
print(Ki(dico_test, 19))

5


In [37]:
print(KiInside(dico_test, communautes, 19))

2


In [11]:
print(QuiDansCommunaute(communautes, 0))

[0, 1]


In [24]:
print(SommeInside(dico_test, communautes, 5))

7


In [18]:
print(MemeCommunaute(communautes, 0, 1))

True


In [41]:
print(SommeTotale(dico_test, communautes, 2))

on aumente la somme car  (8, 3)
1


In [29]:
m=SommePoidsGraphe(dico_test)

In [30]:
print(GainModularite(dico_test, communautes, m, 6, 18))

0.03935986159169551


In [51]:
print(Louvain1(dico_test2))

i: 0 j: 1 gain: -0.026627218934911247
i: 0 j: 2 gain: -0.026627218934911247
i: 0 j: 3 gain: -0.035502958579881644
i: 1 j: 0 gain: -0.026627218934911247
i: 1 j: 2 gain: -0.026627218934911247
i: 1 j: 3 gain: -0.035502958579881644
i: 2 j: 0 gain: -0.026627218934911247
i: 2 j: 1 gain: -0.026627218934911247
i: 2 j: 3 gain: -0.035502958579881644
i: 3 j: 0 gain: -0.035502958579881644
i: 3 j: 1 gain: -0.035502958579881644
i: 3 j: 2 gain: -0.035502958579881644
i: 3 j: 4 gain: -0.04733727810650888
i: 4 j: 5 gain: -0.035502958579881644
i: 4 j: 6 gain: -0.035502958579881644
i: 4 j: 7 gain: -0.035502958579881644
i: 4 j: 3 gain: -0.04733727810650888
i: 5 j: 4 gain: -0.035502958579881644
i: 5 j: 6 gain: -0.026627218934911247
i: 5 j: 7 gain: -0.026627218934911247
i: 6 j: 4 gain: -0.035502958579881644
i: 6 j: 5 gain: -0.026627218934911247
i: 6 j: 7 gain: -0.026627218934911247
i: 7 j: 4 gain: -0.035502958579881644
i: 7 j: 5 gain: -0.026627218934911247
i: 7 j: 6 gain: -0.026627218934911247
[0, 1, 2, 3, 4

à régler